In [97]:
import pandas as pd
from dotenv import load_dotenv
from fastapi import FastAPI
import pyodbc
import os

load_dotenv('.env')

server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
driver = os.getenv('DRIVER')

connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=Yes"
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [98]:
query = '''
            SELECT e.Id, e.Descricao, e.EquipamentoSigmaId FROM Equipamento AS e
            WHERE 
                e.EquipamentoSigmaId is not null
                AND e.Id = 1
                '''
resultado = cursor.execute(query)
registro = resultado.fetchall()
colunas = [column[0] for column in cursor.description]
data = [dict(zip(colunas, row)) for row in registro]
df = pd.DataFrame(data)
df


,Id,Descricao,EquipamentoSigmaId
0,1,Autotransformador INAT7-01 Fase A,30


In [99]:
query = '''
            SELECT 
    oe.TipoObjeto AS TipoOjeto
    , oe.IdentificadorEcm AS IdentificadorModulo
    , oe.Descricao
	, ove.IdentificadorEcm
	, ove.Codigo
	, ove.Descricao
FROM EngineCalculo.ObjetoEcm AS oe
INNER JOIN EngineCalculo.ObjetoVariavelEcm AS ove 
	ON ove.ObjetoId = oe.Id 
WHERE ove.Ativo = 1
ORDER BY oe.TipoObjeto
		, oe.IdentificadorEcm 
		, ove.IdentificadorEcm
                '''
resultado = cursor.execute(query)
registro = resultado.fetchall()
colunas = [column[0] for column in cursor.description]
data = [dict(zip(colunas, row)) for row in registro]
df_2 = pd.DataFrame(data)
df_2[df_2['IdentificadorModulo'] == 194].head(50)

,TipoOjeto,IdentificadorModulo,Descricao,IdentificadorEcm,Codigo
2453,2,194,H2 - Hidrogênio,8492,H2
2454,2,194,CH4 - Metano,8493,CH4
2455,2,194,C2H6 - Etano,8494,C2H6
2456,2,194,C2H4 - Etileno,8495,C2H4
2457,2,194,C2H2 - Acetileno,8496,C2H2
2458,2,194,CO - Monóxido de Carbono,8497,CO
2459,2,194,CO2 - Dióxido de Carbono,8498,CO2
2460,2,194,N2 - Nitrogênio,8499,N2
2461,2,194,O2 -Oxigênio,8500,O2
2462,2,194,Limite de detecção analítica,8501,LimiteAnalitica


In [100]:
df

,Id,Descricao,EquipamentoSigmaId
0,1,Autotransformador INAT7-01 Fase A,30


In [101]:
SUBSYSTE_PARAMS = {
  "Bucha": [
    {"nome": "Capacitância 1", "identificador": 2070},
    {"nome": "Capacitância 2", "identificador": 2071},
    {"nome": "Capacitância 3", "identificador": 2072},
    {"nome": "Tendência de evolução da Capacitância 1", "identificador": 2073},
    {"nome": "Tendência de evolução da Capacitância 2", "identificador": 2074},
    {"nome": "Tendência de evolução da Capacitância 3", "identificador": 2075}, #não tem valor
    {"nome": "Tangente Delta, identificador 1", "identificador": 2076},
    {"nome": "Tangente Delta, identificador 2", "identificador": 2077},
    {"nome": "Tangente Delta, identificador 3", "identificador": 2078},
    {"nome": "Tendência de evolução da Tangente Delta 1", "identificador": 2079},
    {"nome": "Tendência de evolução da Tangente Delta 2", "identificador": 2080},
    {"nome": "Tendência de evolução da Tangente Delta 3", "identificador": 2081},
    {"nome": "Corrente de Fuga, identificador 1", "identificador": 2097},
    {"nome": "Corrente de Fuga, identificador 2", "identificador": 2098},
    {"nome": "Corrente de Fuga, identificador 3", "identificador": 2099},
    {"nome": "Somatória das Correntes de Fuga - BT (baixa tensão), identificador", "identificador": 2187},
    {"nome": "Ângulo da Somatória das Correntes - BT (baixa tensão), identificador", "identificador": 2209},
    {"nome": "Ângulo da Somatória das Correntes - MT (média tensão), identificador", "identificador": 2823}
  ],
  "Parte Ativa": [
    {"nome": "Temperatura do Enrolamento 1", "identificador": 2593},
    {"nome": "Temperatura do Enrolamento 2", "identificador": 2035},
    {"nome": "Temperatura do Enrolamento 3", "identificador": 2043},
    {"nome": "Umidade relativa do óleo", "identificador": 9710},
    {"nome": "Umidade do Papel do Enrolamento 1", "identificador": 11784},
    {"nome": "Umidade do Papel do Enrolamento 2", "identificador": 11787},
    {"nome": "Umidade do Papel do Enrolamento 3", "identificador": 11790},
    {"nome": "Corrente do enrolamento 1", "identificador": 11629},
    {"nome": "Corrente do enrolamento 2", "identificador": 11643},
    {"nome": "Corrente do enrolamento 3", "identificador": 11657},
    {"nome": "Hidrogênio dissolvido no óleo, identificador", "identificador": 2363},
    {"nome": "Tendência de evolução do hidrogênio, identificador", "identificador": 2364},
    {"nome": "H2 - Hidrogênio, identificador", "identificador": 8492},
    {"nome": "CH4 - Metano, identificador", "identificador": 8493},
    {"nome": "C2H6 - Etano:, identificador", "identificador": 8494},
    {"nome": "C2H4 - Etileno, identificador", "identificador": 8495},
    {"nome": "C2H2 - Acetileno, identificador", "identificador": 8496},
    {"nome": "CO - Monóxido de Carbono, identificador", "identificador": 8497},
    {"nome": "CO2 - Dióxido de Carbono, identificador", "identificador": 8498},
    {"nome": "N2 - Nitrogênio, identificador", "identificador": 8499},
    {"nome": "O2 - Oxigênio, identificador", "identificador": 8500}
  ]
}

In [106]:
from datetime import datetime, timedelta

def gerar_datas_intervalo(data_inicial, data_final):
    # Converter as strings de data em objetos datetime
    data_inicial = datetime.strptime(data_inicial, '%Y-%m-%dT%H:%M:%S')
    data_final = datetime.strptime(data_final, '%Y-%m-%dT%H:%M:%S')

    # Inicializar a lista que irá armazenar as datas do intervalo
    datas_intervalo = []

    # Loop para gerar as datas no intervalo
    data_atual = data_inicial
    while data_atual <= data_final:
        datas_intervalo.append(data_atual.strftime('%Y-%m-%dT%H:%M:%S'))
        data_atual += timedelta(days=1)

    return datas_intervalo

# Exemplo de uso da função
data_inicio = '2023-08-01T00:00:00'
data_fim = '2023-08-06T23:59:59'
intervalo_de_datas = gerar_datas_intervalo(data_inicio, data_fim)
print(intervalo_de_datas)


['2023-08-01T00:00:00', '2023-08-02T00:00:00', '2023-08-03T00:00:00', '2023-08-04T00:00:00', '2023-08-05T00:00:00', '2023-08-06T00:00:00']


In [107]:
from src.ecm.ECM import ECM
import datetime


ecm = ECM()
data = ecm.request_results(datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'), datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'), int(df.EquipamentoSigmaId.values[0]))

response<Response [200]>


In [110]:
ecm_list = []
for time in intervalo_de_datas:
    data = ecm.request_results(time,
                                time, 
                                int(df.EquipamentoSigmaId.values[0]))
    ecm_list.append(data)

response<Response [200]>
response<Response [200]>
response<Response [200]>
response<Response [200]>
response<Response [200]>
response<Response [200]>


In [112]:
len(ecm_list)

6

In [108]:
data

[{'equipamentoSigmaId': 30,
  'tipoObjetos': [{'tipoObjeto': 4,
    'objetos': [{'identificador': 101,
      'variaveis': [{'identificador': 1,
        'codigo': 'StatusDevice',
        'valor': '3',
        'dataMedicao': '2022-06-06T02:04:08.767Z',
        'tipoRetorno': 1},
       {'identificador': 2,
        'codigo': 'Autodiag',
        'valor': '0',
        'dataMedicao': '2022-06-06T02:04:08.767Z',
        'tipoRetorno': 1}]},
     {'identificador': 102,
      'variaveis': [{'identificador': 1,
        'codigo': 'StatusDevice',
        'valor': '3',
        'dataMedicao': '2022-06-06T02:04:08.787Z',
        'tipoRetorno': 1},
       {'identificador': 2,
        'codigo': 'Autodiag',
        'valor': '0',
        'dataMedicao': '2022-06-06T02:04:08.787Z',
        'tipoRetorno': 1}]},
     {'identificador': 103,
      'variaveis': [{'identificador': 1,
        'codigo': 'StatusDevice',
        'valor': '3',
        'dataMedicao': '2022-06-06T02:04:08.807Z',
        'tipoRetorno': 

In [84]:
def extract_identifiers(data):
    identifiers = []

    for entry in data:
        for typeObject in entry['tipoObjetos']:
            for data_objetos in typeObject['objetos']:
                for variavel in data_objetos['variaveis']:
                    for bucha_param in jsonTeste['Bucha']:
                        if variavel['identificador'] == bucha_param['identificador'] and 'valor' in variavel:
                            identifiers.append({
                                "Bucha": {
                                "identificador": variavel['identificador'],
                                "codigo": variavel['codigo'],
                                "valor": variavel['valor'],
                                "dataMedicao": variavel['dataMedicao'],
                                "tipoRetorno": variavel['tipoRetorno'],
                                "nome": bucha_param['nome'],
                            }})
                    for bucha_param in jsonTeste['Parte Ativa']:
                        if variavel['identificador'] == bucha_param['identificador'] and 'valor' in variavel:
                            identifiers.append({
                                "Parte Ativa": {
                                "identificador": variavel['identificador'],
                                "codigo": variavel['codigo'],
                                "valor": variavel['valor'],
                                "dataMedicao": variavel['dataMedicao'],
                                "tipoRetorno": variavel['tipoRetorno'],
                                "nome": bucha_param['nome'],
                            }})

    return identifiers

identifiers_array = extract_identifiers(data)

In [85]:
identifiers_array

[{'Parte Ativa': {'identificador': 2035,
   'codigo': 'Get_TemperatureWinding2',
   'valor': '65,5',
   'dataMedicao': '2022-08-17T12:16:10.707Z',
   'tipoRetorno': 1,
   'nome': 'Temperatura do Enrolamento 2'}},
 {'Parte Ativa': {'identificador': 2043,
   'codigo': 'Get_TemperatureWinding3',
   'valor': '64,1',
   'dataMedicao': '2022-08-17T12:16:10.707Z',
   'tipoRetorno': 1,
   'nome': 'Temperatura do Enrolamento 3'}},
 {'Bucha': {'identificador': 2070,
   'codigo': 'Get_FA_Capacitance',
   'valor': '541,5',
   'dataMedicao': '2022-08-17T12:16:10.707Z',
   'tipoRetorno': 1,
   'nome': 'Capacitância 1'}},
 {'Bucha': {'identificador': 2071,
   'codigo': 'Get_FB_Capacitance',
   'valor': '540',
   'dataMedicao': '2022-08-17T12:16:10.707Z',
   'tipoRetorno': 1,
   'nome': 'Capacitância 2'}},
 {'Bucha': {'identificador': 2073,
   'codigo': 'Get_FA_CapacitanceTrend',
   'valor': '0',
   'dataMedicao': '2022-08-17T12:16:10.707Z',
   'tipoRetorno': 1,
   'nome': 'Tendência de evolução da Ca

In [57]:
identifiers_array

[{'identificador': 101, 'nome': None, 'valor': None, 'codigo': None},
 {'identificador': 102, 'nome': None, 'valor': None, 'codigo': None},
 {'identificador': 103, 'nome': None, 'valor': None, 'codigo': None},
 {'identificador': 104, 'nome': None, 'valor': None, 'codigo': None},
 {'identificador': 105, 'nome': None, 'valor': None, 'codigo': None},
 {'identificador': 106, 'nome': None, 'valor': None, 'codigo': None},
 {'identificador': 107, 'nome': None, 'valor': None, 'codigo': None},
 {'identificador': 108, 'nome': None, 'valor': None, 'codigo': None},
 {'identificador': 109, 'nome': None, 'valor': None, 'codigo': None},
 {'identificador': 110, 'nome': None, 'valor': None, 'codigo': None},
 {'identificador': 111, 'nome': None, 'valor': None, 'codigo': None},
 {'identificador': 112, 'nome': None, 'valor': None, 'codigo': None},
 {'identificador': 113, 'nome': None, 'valor': None, 'codigo': None},
 {'identificador': 114, 'nome': None, 'valor': None, 'codigo': None},
 {'identificador': 1

In [ ]:
teste